# LDA Topic Model of MALE Antivax+AntiAntivax Comments on Facebook

In [1]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import nltk

import os
from time import time
from collections import defaultdict

import numpy as np
import pandas as pd

import gensim
from gensim import corpora, utils
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers.dtmmodel import DtmModel

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib
import matplotlib.pyplot as plt

from dateutil import parser
import datetime

import random

%matplotlib inline

In [2]:
import csv

#with open('REDDIT_DATASET_ONE_YEAR_COMMENTS.csv', 'r') as f:
#  reader = csv.reader(f)
#  your_list = list(reader)

# with open('REDDIT_DATASET_ONE_YEAR_COMMENTS_FEMALE_RELATED_COMMENTS.csv', 'r') as f:
#   reader = csv.reader(f)
#   your_list = list(reader)

with open('./data/combined_female_data.csv', 'r') as f:
   reader = csv.reader(f)
   your_list = list(reader)

# reader = pd.read_csv('./data/combined_male_data.csv', sep=',', encoding='latin-1')
# your_list = reader['x'].tolist()
    
your_list.pop(0) # remove the first element which is accidentally put there by R after exporting

['x']

In [3]:
print(your_list[:5]) # first 5 elements
# print(reader[:5]) # first 5 elements

[['Think of the millions of lives he saved!'], ['Glyphosate and GMOs are fulfilling the prophecy of not being able to feed the world coming TRUE through the spurious use of these toxins and bogus science. Genetic engineering and these chemicals go hand in hand to the extent of destruction of sustainability OF LIFE; all broken promises and flawed science are producing such an imbalance disrupting all fauna (GOOD) on earth. Nature is for sale on wall street through patent profiteering of species extinction. \n\nDr. Mercola Interviews Dr. Huber about GMOs\nhttps://youtu.be/yx4UVhJcnpo'], ["Really regular triple antibiotic cream and a bandaid would be cheaper and less time consuming than all of that. And a tetanus shot if not already vaccinated would be easier and less stressful. Then her little Syrus could go play and not be stressed out from this anxiety ridden routine his moms got going on.\n\nEdited to add: my kids pediatrician always says to apply pressure to clot the wound/ stop the 

In [4]:
len(your_list)

307920

In [5]:
import itertools
merged_list = list(itertools.chain(*your_list))

In [6]:
# import random
# merged_list = merged_list[1:10000] # we create a small toy dataset to work with 
# merged_list = random.sample(merged_list, 100000)
# print(merged_list[:5]) # view first 5 elements

In [7]:
len(merged_list)

307920

In [8]:
comments_text = [comment for comment in merged_list]
comments_text_original = [comment for comment in merged_list]

In [9]:
# do word stemming?
from stemming.porter2 import stem

comments_text = [[stem(word) for word in sentence.split(" ")] for sentence in comments_text]
comments_text = [" ".join(sentence) for sentence in comments_text]

In [10]:
# print(comments_text[:5])

## Generate topic model

In this section we generate the topic model. There are some particular parameters that have an important role in shaping the outcome of the model. Changing these will affect the output in various ways. 

I have tested quite a few different combinations and the current output seems to give reasonable results (although we can certainly do better).

num_topics = 30 ... we specify that we want to generate 30 topics from our data
word stemming ... we use a word stemmer to reduce some words to their roots (e.g., walked and walking becomes "walk")
n-grams ... we specify that we want single terms (e.g. "walked"), and also find bi-grams or 2-grams (e.g., "she walked")
stop words ... we remove common English words that are not useful for analysis (e.g. "the", "and")
max_df ... we ignore terms that have a document frequency strictly higher than the given threshold
min_df ... ignore terms that have a document frequency strictly lower than the given threshold


In [11]:
# for good overview of these values see: http://stackoverflow.com/a/35615151/2589495
max_df = 0.05 # ignore terms that have a document frequency strictly higher than x% of docs (or occurs in > n docs if integer value)
min_df = 2 # ignore terms that have a document frequency strictly less than x% of documents (or occurs in < n documents if integer value)
# Some good values found so far...
# max_df = 0.05 (smaller in same order of magnititude is also good e.g. 0.03)
# min_df = 2
# k topics = 50
# passes = 3

# bi-grams:
# tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,ngram_range=(1,2),stop_words='english',analyzer='word')
# unigrams 
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,stop_words='english')
tf = tf_vectorizer.fit_transform(comments_text)

In [12]:
# dictionary.filter_tokens(bad_ids=low_value_words)
# corpus = [dictionary.doc2bow(doc) for doc in texts]
corpus = gensim.matutils.Sparse2Corpus(tf.T)
n_terms, n_docs = corpus.sparse.shape
id2word = {i:word for i, word in enumerate(tf_vectorizer.get_feature_names())}
dictionary = Dictionary.from_corpus(corpus, id2word=id2word)

In [13]:
# generate LDA model
number_topics = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=number_topics, id2word = dictionary, passes=100,update_every=1, chunksize=10000)

In [14]:
# ldamodel.print_topics(-1)

## Visualisation of topics (by term/word)

This is an interactive visualisation of the topic model. The right-hand side shows which terms belong to each topic and how important they are for that topic (in order of importance). 

The left-hand Intertopic Distance Map gives an indication of how closely related the topics are (projected into a two-dimensional space). It may or may not be that helpful analytically, but is included anyway. 

In [15]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis, 'lda_model_FEMALE_antivax_and_antiantivax.html')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [16]:
pyLDAvis.display(vis)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-14 10:49:43.582012. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [17]:
docTopicProbMat = ldamodel[corpus]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:12.030884. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [18]:
docTopicProbMat_list = list(docTopicProbMat)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:12.987555. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [19]:
# print(docTopicProbMat_list[7]) # get the topic probabilities for a document
# print(comments_text[7]) # and the document text

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.641599. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [19]:
comment_probabilities = []
for doc in docTopicProbMat_list:
    docTopic = 999 # reset the topic
    bestMatch = 0 # reset the best match
    possibleMatch = 0 # reset the possible match
    for topic in doc:
        possibleMatch = topic[1]
        if possibleMatch > bestMatch:
                bestMatch = topic[1]
                if bestMatch > 0.6: # minimum threshold for comment topic relatedness
                    docTopic = topic[0]
                # print("bestMatch is now: ",bestMatch)
                # print("docTopic is now: ",docTopic)
    # print("best topic was: ",docTopic)
    comment_probabilities.append(docTopic)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:22.347198. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [20]:
# comment_probabilities[0:10]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:22.949091. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [21]:
comment_probabilities = [probs + 1 for probs in comment_probabilities] # now when we ask for topic n we actually get topic n (not topic n-1)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:23.367263. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [22]:
# print(comments_text[8])
# comment_probabilities[8]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:23.695934. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [23]:
comments_and_topics = pd.DataFrame(
    {'original_comment_text': comments_text_original,
     'comment_text': comments_text,
     'topic': comment_probabilities
    })

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:24.040786. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [24]:
# comments_and_topics

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:24.410738. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [25]:
# a function to get n random example comments assigned to a given topic k
def getTopicExamples(topic_id,num_examples):
    topic_comments = list(comments_and_topics['original_comment_text'][comments_and_topics.topic == topic_id])
    topic_comments = random.sample(topic_comments, num_examples)
    return(topic_comments)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:24.796042. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Extract 10 comment examples for each topic

In this section we extract 50 (randomly selected) text comments that belong to each topic. 

This provides some examples of comments that belong to each topic. The topics are ordered from 1 to k (where k is the number of topics we specified for the model). 

You can use these comment examples to help make sense of the topic terms in the above visualisation, in other words "what is going on" for each topic.

In [26]:
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    print("\n____________________ TOPIC",k,"______________________________\n")
    print(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:25.865608. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



____________________ TOPIC 1 ______________________________

Cierra Raker
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
And she's making MILLIONS by doing this. MILLIONS.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Head. Meet desk. 
FML
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Danielle Patton
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Where was this outbreak? I'm sure the news reports outbreaks like this?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Henri Ellingson
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Was that supposed to be funny?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Bernadett Toth Cahn
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
I believe that her and her brain have had an unconscious uncoupling,...
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Wonder how much compensation is...
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
They don't know gang

In [27]:
f1=open('./example_comments_numComments50_numTopics30_LDA_MODEL_A.txt', 'w+')
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    topic_example_comments = "\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n".join(item for item in topic_example_comments)
    topicheader = "\n____________________ TOPIC",k,"______________________________\n"
    f1.write(str(topicheader))
    # f1.write(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')
    f1.write("\n"+topic_example_comments+"\n")

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:28.425974. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
# theta, _ = ldamodel.inference(corpus) # we can extract the thetas directly from the corpus

In [ ]:
# theta[8] # columns are indexed from 0, so topic n is topic n+1 (e.g. topic22 is topic23)

## Word2vec analysis of text

In this section we do some exploratory text analysis using the novel word2vec algorithm.

This is very experimental... just some computational doodling.

In [28]:
sentences = [comment for comment in merged_list]
sentences = [[stem(word) for word in sentence.split(" ")] for sentence in sentences]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:32.348681. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [29]:
# import nltk
# nltk.download('stopwords')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:32.592061. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [30]:
from nltk.corpus import stopwords
filtered_words = [word for word in sentences if word not in stopwords.words('english')]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:32.814414. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [31]:
# print(filtered_words[2]) # sanity check the data first

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:33.167698. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [32]:
# convert to lowercase
filtered_lower_words = []
for doc in filtered_words:
    filtered_lower_words.append([word.lower() for word in doc])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:33.761164. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [33]:
# remove punctuation
import string
filtered_lower_words_nopunct = []
for doc in filtered_lower_words:
    filtered_lower_words_nopunct.append([''.join(c for c in s if c not in string.punctuation) for s in doc])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:34.243781. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [34]:
# remove empty words due to previous steps 
filtered_lower_words_nopunct_nospaces = []
for doc in filtered_lower_words_nopunct:
    filtered_lower_words_nopunct_nospaces.append([s for s in doc if s])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:34.781583. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
# print(filtered_lower_words_nopunct_nospaces[2]) # sanity check the data quickly

In [35]:
from gensim.models import Word2Vec
min_count = 50
size = 120
window = 7
workers = 6
 
model = Word2Vec(filtered_lower_words_nopunct_nospaces, min_count=min_count, size=size, window=window)


/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:38.431257. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Let's try the classic word2vec example:

King - man + woman = ?

In [36]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:39.628280. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('jone', 0.5967734456062317),
 ('rachel', 0.5753089785575867),
 ('wilson', 0.5662113428115845),
 ('juli', 0.5660696625709534),
 ('fox', 0.5650824308395386),
 ('ami', 0.5457221269607544),
 ('anthoni', 0.5430870652198792),
 ('jane', 0.5403296947479248),
 ('louis', 0.5386220812797546),
 ('miller', 0.5368124842643738)]

Using this kind of analysis we can ask other questions more specific to the paper:
    
vaccine - man + woman = ?

(What is 'vaccine' without taking into account 'men', but taking into account 'women'?)

In [37]:
model.wv.most_similar(positive=['women', 'vaccine'], negative=['men'])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:40.784543. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('vaccin', 0.6421196460723877),
 ('vaccination', 0.6353206038475037),
 ('shot', 0.6214303970336914),
 ('vac', 0.5605137348175049),
 ('dtap', 0.53299880027771),
 ('vaccines', 0.5305996537208557),
 ('tdap', 0.5174212455749512),
 ('vax', 0.5065167546272278),
 ('mmr', 0.5040510892868042),
 ('vaccinations', 0.4990922212600708)]

And we can also do the converse for men:

vaccine - woman + man = ?

(What is 'vaccine' without taking into account women, but taking into account men?)

In [38]:
model.wv.most_similar(positive=['men', 'vaccine'], negative=['women'])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:42.157012. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('shot', 0.5768569111824036),
 ('vaccin', 0.5322495102882385),
 ('shots', 0.5093992948532104),
 ('vac', 0.5052382946014404),
 ('vaccines', 0.5011790990829468),
 ('injection', 0.47435885667800903),
 ('diphtheria', 0.46452999114990234),
 ('vacc', 0.4467175602912903),
 ('vaccination', 0.44401997327804565),
 ('mist', 0.4320049285888672)]

In [39]:
model.most_similar("angry")

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:42.663576. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('furious', 0.8114399909973145),
 ('angri', 0.7423049807548523),
 ('mad', 0.7388920187950134),
 ('disappointed', 0.6522558927536011),
 ('scared', 0.6459805369377136),
 ('annoy', 0.6436542272567749),
 ('frustrat', 0.624653697013855),
 ('uncomfortable', 0.6066946387290955),
 ('happy', 0.602178692817688),
 ('naiv', 0.6005117893218994)]

In [40]:
model.most_similar("autism")

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:43.177164. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('asd', 0.7308833599090576),
 ('autist', 0.6474514007568359),
 ('adhd', 0.6268686056137085),
 ('sids', 0.574080228805542),
 ('epilepsi', 0.552233099937439),
 ('asperger', 0.5371965169906616),
 ('development', 0.5186936259269714),
 ('aspergers', 0.5170971155166626),
 ('diagnosi', 0.5164433121681213),
 ('disabl', 0.5151847004890442)]

In [41]:
model.most_similar(["choice"])

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:10:44.014555. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[('choic', 0.9065365791320801),
 ('decis', 0.685537576675415),
 ('choices', 0.6617480516433716),
 ('decision', 0.6490299701690674),
 ('freedom', 0.6220287680625916),
 ('choos', 0.6026146411895752),
 ('choose', 0.5946564674377441),
 ('rights', 0.5803041458129883),
 ('opinion', 0.575255274772644),
 ('right', 0.5542994737625122)]

We can also see which word within a set of words doesn't belong:

In [44]:
model.wv.doesnt_match("choice science vaccine".split())


/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-15 19:22:58.420165. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


'science'